# Projeto de Analise de Dados de Commodities

## WebScraping dos preços das commodities [Investing.com](https://br.investing.com/commodities/real-time-futures)

In [1]:
from bs4 import BeautifulSoup as bs
import lxml
import requests
import pandas as pd

### Passando parâmetros do site

In [2]:
HEADERS = {"accept":"*/*","user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}
URL = "https://br.investing.com/commodities/real-time-futures"

### "Conectando" (Fazendo o request http) ao site

In [3]:
session = requests.Session()
request = session.get(URL,headers=HEADERS)

## Acessando os dados

In [4]:
## Acessa o html
soup = bs(request.content,'html.parser')
## Acessa a tabela de commodities (Preço)
tag_section = soup.find('table',{'class':'datatable-v2_table__93S4Y'})
## Acessa todas as linhas da tabela
items_table = tag_section.find_all('tr',{'class':'datatable-v2_row__hkEus'})

In [5]:
commodities_data = []
df_preco =[]
for linha in items_table:
    cells = linha.find_all('td')
    if len(cells) > 1:
        nome = cells[1].get_text(strip=True)
        mes = cells[2].get_text(strip=True)
        ultimo_preco = cells[3].get_text(strip=True)
        maximo = cells[4].get_text(strip=True)
        minimo = cells[5].get_text(strip=True)
        variacao = cells[6].get_text(strip=True)
        variacao_pct = cells[7].get_text(strip=True)
        hora = cells[8].get_text(strip=True)

        commodities_data.append({
            "Nome": nome,
            "Mes": mes,
            "Ultimo Preco": ultimo_preco,
            "Maxima": maximo,
            "Minima": minimo,
            "Variacao": variacao,
            "Variacao %": variacao_pct,
            "Hora Preco": hora
        })

In [6]:
df_preco = pd.DataFrame(commodities_data)

## Acessando as opções de tabela

In [7]:
# Localizar os botoes dentro do tablist
botoes_section = soup.find('div', {'class':'bg-white table-browser_table-browser-tabs__q5jip flex overflow-y-scroll mdMax:overflow-x-auto'})
# Acessar o codigo html dos botoes nao acessados e salva-los em uma lista
botoes = botoes_section.find_all('button', {'aria-selected':'false'})
lista_botoes = []
for botao in botoes:
    lista_botoes.append(botao.get_text(strip=True))
print(lista_botoes)

['Desempenho', 'Técnica', 'Especificação']


## Usando Selenium para mudar de guia e atualizar o html

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

###Caminho para o chrome de testes
chrome_options = Options()

### Caminho do chrome driver
service = Service("./chromedriver.exe")
chrome_options.add_argument("user-data-dir=C:/Users/IgorPio/AppData/Local/Google/Chrome/User Data")

# Configura o driver do Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)
# Acessando o site
driver.get("https://br.investing.com/commodities/real-time-futures")


## Automatizando a criação de tabelas

In [9]:
df_desempenho = []
df_tecnica = []
df_especificacao = []
for i in range(0,len(lista_botoes)):
    ### Clicar no botão
    texto = lista_botoes[i]
    ### Usa o WebDriverWait para aguardar que o botão fique visível e então clica
    #botao = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f"//button[@data-test='quote-tab' and contains(text(), '{texto}')]")))
    botao = driver.find_element(By.XPATH, f"//button[@data-test='quote-tab' and contains(text(), '{texto}')]")
    botao.click()
    print(f"Botão '{texto}' foi clicado.")
    ## Esperar um pouco para continuar, para dar tempo de atualizar
    import time
    time.sleep(3)

    ## Acessa o novo html
    novo_html = driver.page_source
    ## Passando para o soup
    soup = bs(novo_html,'html.parser')
    ## Acessa a tabela de commodities (Desempenho)
    tag_section2 = soup.find('table',{'class':'datatable-v2_table__93S4Y'})
    
    ## Acessa todas as linhas da tabela
    items_table = tag_section2.find_all('tr',{'class':'datatable-v2_row__hkEus dynamic-table-v2_row__ILVMx'})
    if i == 0: # Desempenho
        commodities_data = []
        for linha in items_table:
            cells = linha.find_all('td', {'class':'datatable-v2_cell__IwP1U'})
            if len(cells) > 1:
                nome = cells[1].get_text(strip=True)
                diario = cells[2].get_text(strip=True)
                semana1 = cells[3].get_text(strip=True)
                mes1 = cells[4].get_text(strip=True)
                ytd = cells[5].get_text(strip=True)
                ano1 = cells[6].get_text(strip=True)
                ano3 = cells[7].get_text(strip=True)
        
                commodities_data.append({
                    "Nome": nome,
                    "Desempenho Diario": diario,
                    "Semana 1": semana1,
                    "Mes 1": mes1,
                    "YTD": ytd,
                    "Ano 1": ano1,
                    "Anos 3": ano3
                })
        df_desempenho = pd.DataFrame(commodities_data)
        print("Data Desempenho coletada")
        
    elif i == 1: # Tecnica
        commodities_data = []
        for linha in items_table:
            cells = linha.find_all('td', {'class':'datatable-v2_cell__IwP1U'})
            if len(cells) > 1:
                nome = cells[1].get_text(strip=True)
                hora = cells[2].get_text(strip=True)
                diario = cells[3].get_text(strip=True)
                semanal = cells[4].get_text(strip=True)
                mensal = cells[5].get_text(strip=True)
        
                commodities_data.append({
                    "Nome": nome,
                    "Hora": hora,
                    "Tecnica Diaria": diario,
                    "Semanal": semanal,
                    "Mensal": mensal,
                })
        df_tecnica = pd.DataFrame(commodities_data)
        print("Data Tecnica coletada")
    elif i == 2: # Especificacao
        commodities_data = []
        for linha in items_table:
            cells = linha.find_all('td', {'class':'datatable-v2_cell__IwP1U'})
            if len(cells) > 1:
                nome = cells[1].get_text(strip=True)
                codigo = cells[2].get_text(strip=True)
                bolsa = cells[3].get_text(strip=True)
                contrato = cells[4].get_text(strip=True)
                meses = cells[5].get_text(strip=True)
                ponto_base = cells[6].get_text(strip=True)
        
                commodities_data.append({
                    "Nome": nome,
                    "Codigo": codigo,
                    "Bolsa": bolsa,
                    "Contrato": contrato,
                    "Meses": meses,
                    "PontoBase":ponto_base
                })
        df_especificacao = pd.DataFrame(commodities_data)
        print("Data Especificacao coletada")
driver.quit()

Botão 'Desempenho' foi clicado.
Data Desempenho coletada
Botão 'Técnica' foi clicado.
Data Tecnica coletada
Botão 'Especificação' foi clicado.
Data Especificacao coletada


## Mesclar os dataframes com base na coluna nome

In [10]:
df_preco_desempenho = df_preco.merge(df_desempenho, on="Nome", how="outer")
df_pd_tecnica = df_preco_desempenho.merge(df_tecnica, on="Nome", how="outer")
df_final = df_pd_tecnica.merge(df_especificacao, on="Nome", how="outer")

## Limpeza do nome
df_final["Nome"] = df_final["Nome"].str.replace("derived","",regex=False)

In [11]:
# Adiciona o index
df_final.reset_index(inplace=True)
df_final.index +=1
df_final.rename(columns={"index":"ID"}, inplace=True)


## Transforma em tabela excel

### Transformando as tabelas para poder trabalhar com números

In [12]:
df_final.to_excel("DataCommodities.xlsx",index=False)
print("Dados extraidos e combinados, salvos em 'DataCommodities.xlsx'")

Dados extraidos e combinados, salvos em 'DataCommodities.xlsx'


## Inicializa o arquivo Power Bi

In [13]:
import subprocess
path_power_bi = "./AnaliseCommodities.pbix"
subprocess.Popen(['start',path_power_bi], shell=True)

<Popen: returncode: None args: ['start', './AnaliseCommodities.pbix']>